# From Previous Winner
https://github.com/ybabakhin/kaggle_salt_bes_phalanx/blob/master/bes/models/models_zoo.py

In [ ]:
from keras.engine.training import Model
from keras.layers import SpatialDropout2D, Conv2D
from segmentation_models import Unet


def unet_resnext_50(input_shape, freeze_encoder, classes=4):
    resnet_base, hyper_list = Unet(backbone_name='resnext50',
                                   input_shape=input_shape,
                                   input_tensor=None,
                                   encoder_weights='imagenet',
                                   freeze_encoder=freeze_encoder,
                                   skip_connections='default',
                                   decoder_block_type='transpose',
                                   decoder_filters=(128, 64, 32, 16, 8),
                                   decoder_use_batchnorm=True,
                                   n_upsample_blocks=5,
                                   upsample_rates=(2, 2, 2, 2, 2),
                                   classes=classes,
                                   activation='sigmoid')

    x = SpatialDropout2D(0.2)(resnet_base.output)
    x = Conv2D(1, (1, 1), activation="sigmoid", name="prediction")(x)

    model = Model(resnet_base.input, x)

    return model


def unet_resnext_50_lovasz(input_shape, freeze_encoder, classes=4):
    resnet_base, hyper_list = Unet(backbone_name='resnext50',
                                   input_shape=input_shape,
                                   input_tensor=None,
                                   encoder_weights='imagenet',
                                   freeze_encoder=freeze_encoder,
                                   skip_connections='default',
                                   decoder_block_type='transpose',
                                   decoder_filters=(128, 64, 32, 16, 8),
                                   decoder_use_batchnorm=True,
                                   n_upsample_blocks=5,
                                   upsample_rates=(2, 2, 2, 2, 2),
                                   classes=classes,
                                   activation='sigmoid')

    x = SpatialDropout2D(0.2)(resnet_base.output)
    x = Conv2D(1, (1, 1), name="prediction")(x)

    model = Model(resnet_base.input, x)

    return model

# From Simple_U_Net Template

In [ ]:
from keras.utils import multi_gpu_model
from keras.backend.tensorflow_backend import set_session

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
import os
import json

import cv2
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
from keras.losses import mse, binary_crossentropy

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
base_dir = '/data/severstal-steel-defect-detection/'
train_df = pd.read_csv(base_dir+'/train.csv')
train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

print(train_df.shape)
train_df.head()

In [ ]:
mask_count_df = train_df.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.sort_values('hasMask', ascending=False, inplace=True)
print(mask_count_df.shape)
mask_count_df.head()

In [ ]:
sub_df = pd.read_csv(base_dir+'/sample_submission.csv')
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
def build_masks(rles, input_shape):
    depth = len(rles)
    height, width = input_shape
    masks = np.zeros((height, width, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, (width, height))
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]
    
    return rles

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path= base_dir+'train_images',
                 batch_size=32, dim=(256, 1600), n_channels=1,
                 n_classes=4, random_state=2019, shuffle=True, preprocess=None):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        self.preprocess = preprocess
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            img_path = "{}/{}".format(self.base_path, im_name)
            img = self.__load_rgb(img_path)
            
            if self.preprocess: img = self.preprocess(img)
            
            # Store samples
            X[i,] = img

        return X
    
    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            image_df = self.target_df[self.target_df['ImageId'] == im_name]
            
            rles = image_df['EncodedPixels'].values
            masks = build_masks(rles, input_shape=self.dim)
            
            y[i, ] = masks

        return y
    
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img

In [ ]:
base_dir

In [ ]:
img_path = '/data/severstal-steel-defect-detection/train_images/'+train_df.ImageId.values[0]
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


In [ ]:
img.shape

In [ ]:
BATCH_SIZE = 16

train_idx, val_idx = train_test_split(
    mask_count_df.index, random_state=2019, test_size=0.15
)

train_generator = DataGenerator(
    train_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4,
    n_channels=3
)

val_generator = DataGenerator(
    val_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4,
    n_channels=3
)

In [ ]:
def build_model(input_shape, freeze_encoder, classes):
    
    with tf.device('/cpu:0'):
        resnet_base = Unet(
                                   backbone_name='resnext50',
                                   input_shape=input_shape,
#                                    input_tensor=None,
                                   encoder_weights='imagenet',
                                   freeze_encoder=freeze_encoder,
                                   skip_connections='default',
                                   decoder_block_type='transpose',
                                   decoder_filters=(128, 64, 32, 16, 8),
                                   decoder_use_batchnorm=True,
                                   n_upsample_blocks=5,
#                                    upsample_rates=(2, 2, 2, 2, 2),
                                   classes=classes,
                                   activation='sigmoid')

        x = SpatialDropout2D(0.2)(resnet_base.output)
        x = Conv2D(1, (1, 1), activation="sigmoid", name="prediction")(x)

        model = Model(resnet_base.input, x)
    
    parallel_model = multi_gpu_model(model, gpus=8)
    parallel_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
    
    del model
    
    return parallel_model

In [ ]:
model = build_model((256, 1600, 3), True, 4)

In [ ]:
model.summary()

In [ ]:
checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

In [ ]:
history = model.fit_generator(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint],
#     use_multiprocessing=True,
#     workers=4,
    epochs=25,
#     class_weight=class_weights
)

In [ ]:
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing

# LOAD UNET WITH PRETRAINING FROM IMAGENET

with tf.device('/cpu:0'):
    preprocess = get_preprocessing('resnet34') # for resnet, img = (img-110.0)/1.0
    model = Unet('resnet34', input_shape=(256, 1600, 3), classes=4, activation='sigmoid')
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])

parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])

In [ ]:
BATCH_SIZE = 16

train_idx, val_idx = train_test_split(
    mask_count_df.index, random_state=2019, test_size=0.15
)

train_generator = DataGenerator(
    train_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4,
    n_channels=3,
    preprocess=preprocess
)

val_generator = DataGenerator(
    val_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4,
    n_channels=3,
    preprocess=preprocess
)

In [ ]:
checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

In [ ]:
history = parallel_model.fit_generator(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint],
#     use_multiprocessing=True,
#     workers=4,
    epochs=25,
#     class_weight=class_weights
)